In [1]:
import torch
import torch.nn as nn
import FrEIA.framework as Ff
import FrEIA.modules as Fm
import pandas as pd
import torch.distributions as dist
import matplotlib.pyplot as plt
import numpy as np
import matplotlib

from helpers import *
from sklearn.preprocessing import StandardScaler
from os.path import exists
from argparse import ArgumentParser

matplotlib.use('Agg')
plt.rcParams.update({'axes.labelsize': 14})

In [2]:
parser = ArgumentParser()
parser.add_argument("-c", "--cuda", dest="cuda", default=0, type=int, help='Cuda number')

parser.add_argument("-f", '--flows', dest="flows", default=8, type=int, help='Number of flows')
parser.add_argument("--nn-hidden", dest="nn_hidden", default=3, type=int, help='Number of hidden layers for the NN')
parser.add_argument("--nn-nodes", dest="nn_nodes", default=200, type=int, help='Number of hidden nodes for NN')
parser.add_argument("--model", dest="model", default='8flows_3layer_200nodes_50000batch/model.pt', type=str, help='Path to model')
parser.add_argument("--output", dest="output", default='eval_dummy/', type=str, help='Path for output files')

parser.add_argument("--test", dest="test", default=1, type=int, help='Test run {True, False}')

_StoreAction(option_strings=['--test'], dest='test', nargs=None, const=None, default=1, type=<class 'int'>, choices=None, help='Test run {True, False}', metavar=None)

In [3]:
# pass default arguments if executed as ipynb
try: 
    if get_ipython().__class__.__name__ == 'ZMQInteractiveShell': args = parser.parse_args("") 
except:
    args = parser.parse_args()

print(args)
ensure_dir(args.output)

Namespace(cuda=0, flows=8, model='8flows_3layer_200nodes_50000batch/model.pt', nn_hidden=3, nn_nodes=200, output='eval_dummy/', test=1)


In [4]:
if exists('/ceph/lsowa/recoil/dt.root'):
    dfdata = load_from_root('/ceph/lsowa/recoil/dt.root', test=args.test)
    dfmc = load_from_root('/ceph/lsowa/recoil/mc.root', test=args.test)
else:
    # when running on cluster
    dfdata = load_from_root('recoil/dt.root', test=args.test)
    dfmc = load_from_root('recoil/mc.root', test=args.test)

Reading files from:  /ceph/lsowa/recoil/dt.root
No.  1
Reading files from:  /ceph/lsowa/recoil/mc.root
No.  1


In [5]:
pz = dist.MultivariateNormal(torch.zeros(2), torch.eye(2))

In [6]:
cond = ['metphi','pt_vis_c', 'phi_vis_c','pt_1', 'pt_2','dxy_1', 'dxy_2','dz_1',
        'dz_2','eta_1', 'eta_2','mass_1', 'mass_2','metSumEt']
names = ['uP1_uncorrected', 'uP2_uncorrected']

In [7]:
data = dfdata[names].to_numpy().astype(float)
mc = dfmc[names].to_numpy().astype(float)
cdata = dfdata[cond].to_numpy().astype(float)
cmc = dfmc[cond].to_numpy().astype(float)

In [8]:
# +
# Z standardize inputs
# +
input_scaler = StandardScaler()
data = input_scaler.fit_transform(data)
mc = input_scaler.transform(mc)

cond_scaler = StandardScaler()
cdata = cond_scaler.fit_transform(cdata)
cmc = cond_scaler.transform(cmc)

In [9]:
data, mc, cdata, cmc = torch.tensor(data), torch.tensor(mc), torch.tensor(cdata), torch.tensor(cmc)

In [10]:
# Setup Model

def mlp_constructor(input_dim=2, out_dim=2, hidden_nodes=args.nn_nodes):
    
    layers = [nn.Linear(input_dim, hidden_nodes), nn.ReLU()]
    for n in range(args.nn_hidden-1):
        layers.append(nn.Linear(hidden_nodes, hidden_nodes))
        layers.append(nn.ReLU())
    layers.append(nn.Linear(hidden_nodes, out_dim))
    
    model = nn.Sequential(*layers)
    return model

model = Ff.SequenceINN(2)
for k in range(args.flows):
    model.append(Fm.RNVPCouplingBlock, subnet_constructor=mlp_constructor, 
                    clamp=2, cond=0, cond_shape=(cmc.shape[1],))

In [11]:
model.load_state_dict(torch.load(args.model, map_location=torch.device('cpu')))

<All keys matched successfully>

In [ ]:
def density_2d(hist, line, line_label='gaussian', hist_label='model(data)', 
                xlim = [-3, 3], ylim = [-3, 3], save_as=None, xlabel=r'$u_\perp$',
                crosses = None, crosses_label = None, crosses_color='red',
                ylabel=r'$u_\parallel$', gridsize=(40,40), bins=100, levels=4, alpha=0.7):
    
    fig, ax = plt.subplots(2, 2, figsize=(8, 8), gridspec_kw={'width_ratios': [2, 1],
                                                                'height_ratios': [1, 2]})
    plt.subplots_adjust(hspace=0, wspace=0)


    # heatmap
    ax[1,0].hexbin(x=hist[:,0], y=hist[:,1], extent= xlim + ylim,
                    label=hist_label, gridsize=gridsize, cmap='Blues', edgecolors=None)
    sns.kdeplot(x=line[:,0], y=line[:,1], shade=False, 
                ax=ax[1,0], color='orange', levels=levels, alpha=alpha)
    ax[1,0].plot([], [], '-', label=line_label, color='orange')
    if crosses is not None:
        ax[1,0].plot(crosses[0], crosses[1], '+', label=crosses_label, color=crosses_color)
    ax[1,0].set_ylim(ylim)
    ax[1,0].set_xlim(xlim)
    ax[1,0].set_xlabel(xlabel)
    ax[1,0].set_ylabel(ylabel)
    ax[1,0].legend(loc='lower left', bbox_to_anchor=(1,1))
    
    # x axis
    _ = ax[0,0].hist(hist[:,0], bins=bins, density=True, 
                        range=[xlim[0], xlim[1]], label=hist_label)
    _ = ax[0,0].hist(line[:,0], bins=bins, range=[xlim[0], xlim[1]],
                        density=True, histtype=u'step', label=line_label)
    ax[0,0].set_xticks([])
    ax[0,0].set_yticks([])
    ax[0,0].set_ylabel('a.u.')
    ax[0,0].set_xlim(xlim)

    # y axis
    _ = ax[1,1].hist(hist[:,1], bins=bins, density=True, 
                        label=hist_label,range=[ylim[0], ylim[1]], 
                        orientation='horizontal')
    _ = ax[1,1].hist(line[:,1], bins=bins, density=True, range=[ylim[0], ylim[1]],
                        histtype=u'step', label=line_label, orientation='horizontal')
    ax[1,1].set_xticks([])
    ax[1,1].set_yticks([])
    ax[1,1].set_xlabel('a.u.')
    ax[1,1].set_ylim(ylim)

    # third wheel
    _ = ax[0,1].axis('off')
    
    if save_as is not None:
        plt.savefig(save_as)
        plt.clf()
    # -

In [16]:
model.cpu()
z, _ = evaluate_sequential(model, data.float(), cond=cdata.float())
gaussian = pz.sample((100000, ))
density_2d(z.cpu().detach().numpy(), gaussian.cpu().detach().numpy(), 
            line_label=r'target gaussian $z$', hist_label=r'model(y, $\mathrm{cond}_\mathrm{Data}$)= $\hat{z}$', 
            xlim = [-3, 3], ylim = [-3, 3], save_as=args.output+'2d_gaussian_data.pdf')

In [ ]:
xxxxxxxxxxxxxxxxxxx

In [ ]:
z, _ = evaluate_sequential(model, mc.float(), cond=cmc.float())
gaussian = pz.sample((100000, ))
density_2d(z.cpu().detach().numpy(), gaussian.cpu().detach().numpy(), 
            line_label=r'target gaussian $z$', hist_label=r'model(y, $\mathrm{cond}_\mathrm{MC}$)= $\hat{z}$', 
            xlim = [-3, 3], ylim = [-3, 3], save_as=args.output+'2d_gaussian_mc.pdf')

/ceph/lsowa/envs/cluster/lib/python3.6/site-packages/seaborn/distributions.py:1718: UserWarning: `shade_lowest` is now deprecated in favor of `thresh`. Setting `thresh=0.05`, but please update your code.
  warnings.warn(msg, UserWarning)


In [ ]:

# Predict y

z = pz.sample((cmc.shape[0], ))
u, _ = evaluate_sequential(model, z, cmc.float(), rev=True)

u = u.cpu().detach().numpy()
u = input_scaler.inverse_transform(u)

data = input_scaler.inverse_transform(data)
mc = input_scaler.inverse_transform(mc)


In [ ]:
density_2d(u, data, 
            line_label=r'$u^\mathrm{MC}$', hist_label=r'model(z, $\mathrm{cond}_\mathrm{Data}$)=$\hat{u}$', 
            xlim = [-160, 100], ylim = [-80, 30], save_as=args.output+'2d_comp_umc_to_data.pdf')

/ceph/lsowa/envs/cluster/lib/python3.6/site-packages/seaborn/distributions.py:1718: UserWarning: `shade_lowest` is now deprecated in favor of `thresh`. Setting `thresh=0.05`, but please update your code.
  warnings.warn(msg, UserWarning)


In [ ]:
density_2d(u, mc, 
            line_label=r'$u^\mathrm{MC}$', hist_label=r'model(z, $\mathrm{cond}_\mathrm{MC}$)=$\hat{u}$', 
            xlim = [-160, 100], ylim = [-80, 30], save_as=args.output+'2d_comp_umc_to_mc.pdf')

/ceph/lsowa/envs/cluster/lib/python3.6/site-packages/seaborn/distributions.py:1718: UserWarning: `shade_lowest` is now deprecated in favor of `thresh`. Setting `thresh=0.05`, but please update your code.
  warnings.warn(msg, UserWarning)


# ### Compare MC -> Data

In [ ]:
# u parallel
interval = [-170, 100]
_ = plt.hist(u[:,0], density=True, bins=100, range=interval, label=r'model(z,$c^\mathrm{MC}$)=$u_\parallel$')
_ = plt.hist(data[:,0], histtype=u'step', density=True, bins=100, 
                range=interval, linewidth=2, color='black', label=r'$u^\mathrm{Data}_\parallel$')
_ = plt.hist(dfmc['uP2_uncorrected'].values, histtype=u'step', density=True, bins=100, 
                range=interval, linewidth=2, color='red', label=r'$u^\mathrm{MC}_\perp $ uncorrected')
plt.xlabel(r'$u_\perp$')
plt.ylabel('a. u.')
plt.legend()
plt.savefig(args.output+'u_perp.pdf')
plt.clf()

In [ ]:
# response
response(uperp=u[:,0], ptz=dfmc['pt_vis_c'], 
            save_path=args.output+'response.pdf', 
            cut_max=200, cut_min=25)

n, bins, edges = plt.hist(up, histtype=r'step', bins=200, range=[-200, 200], 
                            density=True, label=r'$\mathrm{u}_\parallel$')
n1, bins1, edges1 = plt.hist(ptz, histtype=r'step', bins=200, range=[-200, 200], 
                                density=True, label=r'$p_\mathrm{T}^Z}\rangle$')
plt.xlim([-200, 200])
plt.legend()
plt.xlabel('GeV')
plt.ylabel('a.u.')
plt.savefig(args.output+'uperp_ptw.pdf')
plt.clf()

In [ ]:
csingle_event = torch.concat([cmc[100,:].unsqueeze(0)]*10000)

In [ ]:
z = pz.sample((csingle_event.shape[0], ))
u, _ = evaluate_sequential(model, z, cond=csingle_event.float(), rev=True)

u = input_scaler.inverse_transform(u)

In [ ]:
density_2d(u, mc, 
            line_label=r'target $y_\mathrm{MC}$', hist_label=r'model(z, $\mathrm{cond}^\mathrm{fixed}_\mathrm{MC}$)=$\hat{y}$', 
            xlim = [-160, 100], ylim = [-80, 30], save_as=args.output+'fixedevent.pdf')

/ceph/lsowa/envs/cluster/lib/python3.6/site-packages/seaborn/distributions.py:1718: UserWarning: `shade_lowest` is now deprecated in favor of `thresh`. Setting `thresh=0.05`, but please update your code.
  warnings.warn(msg, UserWarning)


In [ ]:
z = pz.sample((cdata[:100000,:].shape[0], ))
layerwise2d(model, z, cond=cdata[:100000,:], save_path=args.output+'layerwise.pdf', xlim=[-150, 100], ylim=[-80, 30], input_scaler=input_scaler)

/work/lsowa/recoil_correction/helpers.py:123: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  cond = torch.tensor(cond)


In [ ]:
for cond_no, cond_name in enumerate(cond):
    condition_correlation(model=model,
                            cond=cdata,
                            cond_no=cond_no,
                            deltas=np.linspace(-3, 3, 21),
                            input_scaler=input_scaler,
                            save_path=args.output+'cond_scan_' + cond_name + '.pdf',
                            cond_name= r'$\Delta$',
                            xlim=[-100, 80],
                            device=torch.device(args.cuda),
                            title = cond_name + r'+$\Delta\cdot \sigma_{}$'.format("{"+cond_name+"}"))
    print(cond_name, 'done')